In [1]:
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import numpy as np

# Para mostrar todas las columnas cuando se imprime un df
pd.set_option('display.max_columns', None)
plt.style.use('ggplot')

# Cadenas de Markov para melodías con armonía tradicional

## Duración fija de notas y silencios

In [148]:
import numpy as np
import random

# Definir la escala de Do mayor (una octava)
notes = ['C', 'D', 'E', 'F', 'G', 'A', 'B', 'C_alto']

# Matriz de transición de ejemplo (probabilidades uniformes por simplicidad)
matriz_transicion = np.full((len(notes), len(notes)), 1/len(notes))

def generate_melody(num_notas, duracion_nota, matriz_transicion):
    melody = []
    current_note = random.choice(notes)  # Nota inicial

    for _ in range(num_notas):
        melody.append((current_note, duracion_nota))
        current_note_index = notes.index(current_note)
        current_note = np.random.choice(notes, p=matriz_transicion[current_note_index])
    
    return melody

# Ejemplo de uso
num_notas = 10
duracion_nota = 0.5  # En segundos
melody = generate_melody(num_notas, duracion_nota, matriz_transicion)
melody


[('G', 0.5),
 ('F', 0.5),
 ('D', 0.5),
 ('F', 0.5),
 ('B', 0.5),
 ('C', 0.5),
 ('C_alto', 0.5),
 ('D', 0.5),
 ('C_alto', 0.5),
 ('A', 0.5)]

In [163]:
from pydub import AudioSegment
from pydub.generators import Sine

def note_to_freq(nota):
    """ Convertir la nota musical a su frecuencia. """
    frecuencias_notas = {
        'C': 261.63, 'D': 293.66, 'E': 329.63, 'F': 349.23, 
        'G': 392.00, 'A': 440.00, 'B': 493.88, 'C_alto': 523.25
    }
    return frecuencias_notas[nota]

def generate_melody(num_notas, duracion_nota, matriz_transicion):
    """ Generar una melodía basada en la cadena de Markov. """
    melody = []
    current_note = random.choice(notes)  # Nota inicial

    for _ in range(num_notas):
        melody.append((current_note, duracion_nota))
        current_note_index = notes.index(current_note)
        current_note = np.random.choice(notes, p=matriz_transicion[current_note_index])
    
    return melody

def melody_to_audio(melodia):
    """ Convertir una melodía en un segmento de audio. """
    song = AudioSegment.silent(duration=0)
    for nota, duracion in melodia:
        frecuencia = note_to_freq(nota)
        segmento_nota = Sine(frecuencia).to_audio_segment(duration=int(duracion * 1000))
        song += segmento_nota

    return song

# Generar la melodía
num_notas = 10
duracion_nota = 0.2  # en segundos
melodia = generate_melody(num_notas, duracion_nota, matriz_transicion)

# Convertir la melodía en audio
audio = melody_to_audio(melodia)

# Exportar a un archivo WAV
archivo_salida = "audios/1.melodia_sin_silencios.wav"
audio.export(archivo_salida, format="wav")
archivo_salida


'audios/1.melodia_sin_silencios.wav'

# Introduciendo silencios

In [157]:
note_frequencies = {
    'C': 261.63, 'D': 293.66, 'E': 329.63, 'F': 349.23, 
    'G': 392.00, 'A': 440.00, 'B': 493.88, 'C_high': 523.25,
    'Rest': 0  # La frecuencia para el silencio es 0
}
def note_to_freq(nota):
    """ Convertir la nota musical a su frecuencia. Ajustado para el silencio. """
    return note_frequencies[nota]

# Actualizar notas y matriz de transición para incluir silencios
notes_with_rest = notes + ['Rest']
transition_matrix_with_rest = np.full((len(notes_with_rest), len(notes_with_rest)), 1/len(notes_with_rest))
pd.DataFrame(transition_matrix_with_rest, index=note_frequencies.keys(), columns=note_frequencies.keys())

,C,D,E,F,G,A,B,C_high,Rest
C,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111
D,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111
E,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111
F,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111
G,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111
A,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111
B,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111
C_high,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111
Rest,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111


In [158]:
def generate_melody_with_rests(notas, duracion_nota, matriz_transicion, num_notas=10):
    """ Generar una melodía con silencios basada en la cadena de Markov. """
    melody = []
    current_note = random.choice(notas)  # Nota inicial
    for _ in range(num_notas):
        melody.append((current_note, duracion_nota))
        current_note_index = notas.index(current_note)
        # Para escoger del renglón en el que estamos
        current_note = np.random.choice(notas, p=matriz_transicion[current_note_index]) 
    
    return melody

duracion_nota=1

melodia_con_silencios = generate_melody_with_rests(notes_with_rest, duracion_nota, transition_matrix_with_rest)
melodia_con_silencios


[('C', 1),
 ('Rest', 1),
 ('C_alto', 1),
 ('Rest', 1),
 ('A', 1),
 ('C_alto', 1),
 ('A', 1),
 ('E', 1),
 ('A', 1),
 ('A', 1)]

In [169]:
def melodia_rests(melody):
    """ Convert a melody with rests to an audio segment. """
    song = AudioSegment.silent(duration=0)
    for note, duration in melody:
        if note == 'Rest':
            note_segment = AudioSegment.silent(duration=int(duration * 1000))
        else:
            frequency = note_to_freq(note)
            note_segment = Sine(frequency).to_audio_segment(duration=int(duration * 1000))
        song += note_segment

    return song



In [175]:
def melody_to_audio_with_rests(melodia):
    """ Convertir una melodía con silencios en un segmento de audio. """
    song = AudioSegment.silent(duration=0)
    for nota, duracion in melodia:
        if nota == 'Rest':
            note_segment = AudioSegment.silent(duration=int(duracion * 1000))
        else:
            frecuencia = note_to_freq(nota)
            note_segment = Sine(frecuencia).to_audio_segment(duration=int(duracion * 1000))
        song += note_segment

    return song


duracion_nota=.5



# Convertir la melodía con silencios en audio
melodia_con_silencios = generate_melody_with_rests(notes_with_rest, duracion_nota, transition_matrix_with_rest)
audio_con_silencios = melodia_rests(melodia_con_silencios)

# Exportar a un archivo WAV
archivo_salida_con_silencios = "./audios/2.melodia_con_silencios.wav"
audio_con_silencios.export(archivo_salida_con_silencios, format="wav")
archivo_salida_con_silencios


'./audios/2.melodia_con_silencios.wav'

## Duración variable de notas

## 1.expandiendo el espacio de estados

In [27]:
import numpy as np
import random
from pydub import AudioSegment
from pydub.generators import Sine

# Define the notes and durations
notes = ['C', 'D', 'E', 'F', 'G', 'A', 'B', 'C_high']
durations = ['quarter', 'eighth', 'sixteenth']  # Durations as fractions of a whole note
states = [note + '-' + duration for note in notes for duration in durations]

# Example transition matrix with uniform probabilities (this can be customized)
transition_matrix = np.full((len(states), len(states)), 1/len(states))

def note_to_freq(note):
    """ Convert musical note to its frequency. """
    note_frequencies = {
        'C': 261.63, 'D': 293.66, 'E': 329.63, 'F': 349.23, 
        'G': 392.00, 'A': 440.00, 'B': 493.88, 'C_high': 523.25
    }
    return note_frequencies.get(note, 0)  # Return 0 if note not found

def duration_to_time(duration, quarter_note_length):
    """ Convert duration type to actual time length. """
    length_map = {
        'quarter': quarter_note_length,
        'eighth': quarter_note_length / 2,
        'sixteenth': quarter_note_length / 4
    }
    return length_map.get(duration, 0)  # Return 0 if duration not found

def generate_melody(num_bars, quarter_note_length, transition_matrix):
    """ Generate a melody within the specified number of bars. """
    melody = []
    total_duration = 0
    bar_length = 4 * quarter_note_length  # Total length of a bar in 4/4 time

    while total_duration < num_bars * bar_length:
        current_state = random.choice(states)  # Starting state
        note, duration = current_state.split('-')
        note_length = duration_to_time(duration, quarter_note_length)

        # Check if adding this note exceeds the bar length
        if total_duration + note_length <= num_bars * bar_length:
            melody.append((note, note_length))
            total_duration += note_length

            current_state_index = states.index(current_state)
            current_state = np.random.choice(states, p=transition_matrix[current_state_index])

    return melody



# Example usage
num_bars = 4
quarter_note_length = 0.5  # Length of a quarter note in seconds
melody = generate_melody(num_bars, quarter_note_length, transition_matrix)

melody


[('B', 0.25),
 ('A', 0.25),
 ('G', 0.125),
 ('C', 0.5),
 ('C', 0.5),
 ('B', 0.5),
 ('G', 0.125),
 ('E', 0.125),
 ('F', 0.125),
 ('C', 0.5),
 ('E', 0.125),
 ('D', 0.25),
 ('E', 0.5),
 ('A', 0.5),
 ('D', 0.125),
 ('C', 0.125),
 ('F', 0.5),
 ('D', 0.25),
 ('C', 0.125),
 ('E', 0.5),
 ('G', 0.5),
 ('C', 0.5),
 ('F', 0.125),
 ('E', 0.125),
 ('B', 0.125),
 ('A', 0.125),
 ('F', 0.125),
 ('C', 0.25),
 ('B', 0.125)]

In [28]:
def melody_to_audio(melody):
    """ Convert a melody to an audio segment. """
    song = AudioSegment.silent(duration=0)
    for note, duration in melody:
        frequency = note_to_freq(note)
        note_segment = Sine(frequency).to_audio_segment(duration=int(duration * 1000))
        song += note_segment

    return song

# Convert the melody to audio
audio = melody_to_audio(melody)

# Export to a WAV file
output_file = "./audios/variable_length_melody.wav"
audio.export(output_file, format="wav")
output_file

'./audios/variable_length_melody.wav'

# Tomando más que el pasado inmediato

Una nota, bigramas, ngramas

In [52]:
def create_bigram_states(notes, durations):
    """ Create bigram states with each element being a note-duration pair. """
    bigram_states = [('sos', note + '-' + duration) for note in notes for duration in durations]
    bigram_states += [(note1 + '-' + duration1, note2 + '-' + duration2) 
                      for note1 in notes for duration1 in durations 
                      for note2 in notes for duration2 in durations]
    bigram_states.append(('eos', 'eos'))  # Add end of song state
    return bigram_states

duration_lengths={'quarter': 1, 'eighth': 0.5, 'sixteenth': 0.25}

# Create bigram states
bigram_states = create_bigram_states(notes, durations)
bigram_states

[('sos', 'C-quarter'),
 ('sos', 'C-eighth'),
 ('sos', 'C-sixteenth'),
 ('sos', 'D-quarter'),
 ('sos', 'D-eighth'),
 ('sos', 'D-sixteenth'),
 ('sos', 'E-quarter'),
 ('sos', 'E-eighth'),
 ('sos', 'E-sixteenth'),
 ('sos', 'F-quarter'),
 ('sos', 'F-eighth'),
 ('sos', 'F-sixteenth'),
 ('sos', 'G-quarter'),
 ('sos', 'G-eighth'),
 ('sos', 'G-sixteenth'),
 ('sos', 'A-quarter'),
 ('sos', 'A-eighth'),
 ('sos', 'A-sixteenth'),
 ('sos', 'B-quarter'),
 ('sos', 'B-eighth'),
 ('sos', 'B-sixteenth'),
 ('sos', 'C_high-quarter'),
 ('sos', 'C_high-eighth'),
 ('sos', 'C_high-sixteenth'),
 ('C-quarter', 'C-quarter'),
 ('C-quarter', 'C-eighth'),
 ('C-quarter', 'C-sixteenth'),
 ('C-quarter', 'D-quarter'),
 ('C-quarter', 'D-eighth'),
 ('C-quarter', 'D-sixteenth'),
 ('C-quarter', 'E-quarter'),
 ('C-quarter', 'E-eighth'),
 ('C-quarter', 'E-sixteenth'),
 ('C-quarter', 'F-quarter'),
 ('C-quarter', 'F-eighth'),
 ('C-quarter', 'F-sixteenth'),
 ('C-quarter', 'G-quarter'),
 ('C-quarter', 'G-eighth'),
 ('C-quarter', '

In [41]:
# Simplified transition matrix for bigram states (uniform probabilities for demonstration)
transition_matrix_bigrams = np.full((len(bigram_states), len(bigram_states)), 1/len(bigram_states))
pd.DataFrame(transition_matrix_bigrams,index=bigram_states,columns=bigram_states)

,"(sos, C-quarter)","(sos, C-eighth)","(sos, C-sixteenth)","(sos, D-quarter)","(sos, D-eighth)","(sos, D-sixteenth)","(sos, E-quarter)","(sos, E-eighth)","(sos, E-sixteenth)","(sos, F-quarter)","(sos, F-eighth)","(sos, F-sixteenth)","(sos, G-quarter)","(sos, G-eighth)","(sos, G-sixteenth)","(sos, A-quarter)","(sos, A-eighth)","(sos, A-sixteenth)","(sos, B-quarter)","(sos, B-eighth)","(sos, B-sixteenth)","(sos, C_high-quarter)","(sos, C_high-eighth)","(sos, C_high-sixteenth)","(C-quarter, C-quarter)","(C-quarter, C-eighth)","(C-quarter, C-sixteenth)","(C-quarter, D-quarter)","(C-quarter, D-eighth)","(C-quarter, D-sixteenth)","(C-quarter, E-quarter)","(C-quarter, E-eighth)","(C-quarter, E-sixteenth)","(C-quarter, F-quarter)","(C-quarter, F-eighth)","(C-quarter, F-sixteenth)","(C-quarter, G-quarter)","(C-quarter, G-eighth)","(C-quarter, G-sixteenth)","(C-quarter, A-quarter)","(C-quarter, A-eighth)","(C-quarter, A-sixteenth)","(C-quarter, B-quarter)","(C-quarter, B-eighth)","(C-quarter, B-sixteenth)","(C-quarter, C_high-quarter)","(C-quarter, C_high-eighth)","(C-quarter, C_high-sixteenth)","(C-eighth, C-quarter)","(C-eighth, C-eighth)","(C-eighth, C-sixteenth)","(C-eighth, D-quarter)","(C-eighth, D-eighth)","(C-eighth, D-sixteenth)","(C-eighth, E-quarter)","(C-eighth, E-eighth)","(C-eighth, E-sixteenth)","(C-eighth, F-quarter)","(C-eighth, F-eighth)","(C-eighth, F-sixteenth)","(C-eighth, G-quarter)","(C-eighth, G-eighth)","(C-eighth, G-sixteenth)","(C-eighth, A-quarter)","(C-eighth, A-eighth)","(C-eighth, A-sixteenth)","(C-eighth, B-quarter)","(C-eighth, B-eighth)","(C-eighth, B-sixteenth)","(C-eighth, C_high-quarter)","(C-eighth, C_high-eighth)","(C-eighth, C_high-sixteenth)","(C-sixteenth, C-quarter)","(C-sixteenth, C-eighth)","(C-sixteenth, C-sixteenth)","(C-sixteenth, D-quarter)","(C-sixteenth, D-eighth)","(C-sixteenth, D-sixteenth)","(C-sixteenth, E-quarter)","(C-sixteenth, E-eighth)","(C-sixteenth, E-sixteenth)","(C-sixteenth, F-quarter)","(C-sixteenth, F-eighth)","(C-sixteenth, F-sixteenth)","(C-sixteenth, G-quarter)","(C-sixteenth, G-eighth)","(C-sixteenth, G-sixteenth)","(C-sixteenth, A-quarter)","(C-sixteenth, A-eighth)","(C-sixteenth, A-sixteenth)","(C-sixteenth, B-quarter)","(C-sixteenth, B-eighth)","(C-sixteenth, B-sixteenth)","(C-sixteenth, C_high-quarter)","(C-sixteenth, C_high-eighth)","(C-sixteenth, C_high-sixteenth)","(D-quarter, C-quarter)","(D-quarter, C-eighth)","(D-quarter, C-sixteenth)","(D-quarter, D-quarter)","(D-quarter, D-eighth)","(D-quarter, D-sixteenth)","(D-quarter, E-quarter)","(D-quarter, E-eighth)","(D-quarter, E-sixteenth)","(D-quarter, F-quarter)","(D-quarter, F-eighth)","(D-quarter, F-sixteenth)","(D-quarter, G-quarter)","(D-quarter, G-eighth)","(D-quarter, G-sixteenth)","(D-quarter, A-quarter)","(D-quarter, A-eighth)","(D-quarter, A-sixteenth)","(D-quarter, B-quarter)","(D-quarter, B-eighth)","(D-quarter, B-sixteenth)","(D-quarter, C_high-quarter)","(D-quarter, C_high-eighth)","(D-quarter, C_high-sixteenth)","(D-eighth, C-quarter)","(D-eighth, C-eighth)","(D-eighth, C-sixteenth)","(D-eighth, D-quarter)","(D-eighth, D-eighth)","(D-eighth, D-sixteenth)","(D-eighth, E-quarter)","(D-eighth, E-eighth)","(D-eighth, E-sixteenth)","(D-eighth, F-quarter)","(D-eighth, F-eighth)","(D-eighth, F-sixteenth)","(D-eighth, G-quarter)","(D-eighth, G-eighth)","(D-eighth, G-sixteenth)","(D-eighth, A-quarter)","(D-eighth, A-eighth)","(D-eighth, A-sixteenth)","(D-eighth, B-quarter)","(D-eighth, B-eighth)","(D-eighth, B-sixteenth)","(D-eighth, C_high-quarter)","(D-eighth, C_high-eighth)","(D-eighth, C_high-sixteenth)","(D-sixteenth, C-quarter)","(D-sixteenth, C-eighth)","(D-sixteenth, C-sixteenth)","(D-sixteenth, D-quarter)","(D-sixteenth, D-eighth)","(D-sixteenth, D-sixteenth)","(D-sixteenth, E-quarter)","(D-sixteenth, E-eighth)","(D-sixteenth, E-sixteenth)","(D-sixteenth, F-quarter)","(D-sixteenth, F-eighth)","(D-sixteenth, F-sixteenth)","(D-sixteenth, G-quarter)","(D-sixteenth, G-eighth)","(D-sixteenth, G-sixteen

In [53]:
def generate_melody_bigrams(num_bars, quarter_note_length, transition_matrix, bigram_states):
    """ Generate a melody considering the previous two notes (bigrams). """
    melody = []
    total_duration = 0
    current_state = ('sos', random.choice(notes) + '-' + random.choice(durations))

    while total_duration < num_bars * 4 and current_state[0] != 'eos':  # 4/4 time
        # Extract current note and duration
        _, current_note_duration = current_state
        note, duration = current_note_duration.split('-')
        melody.append((note, duration_lengths[duration] * quarter_note_length))
        total_duration += duration_lengths[duration]

        # Transition to next state
        current_state_index = bigram_states.index(current_state)
        current_state = bigram_states[np.random.choice(np.arange(len(bigram_states)), p=transition_matrix[current_state_index])]
    
    return melody

# Generate the melody with bigrams
num_bars = 4
quarter_note_length = 500  # Quarter note length in milliseconds (0.5 seconds)
melody_bigrams = generate_melody_bigrams(num_bars, quarter_note_length, transition_matrix_bigrams, bigram_states)

melody_bigrams  # Displaying the generated melody (as note-duration pairs)

[('A', 250.0),
 ('F', 125.0),
 ('B', 250.0),
 ('C_high', 125.0),
 ('B', 125.0),
 ('B', 500),
 ('C', 125.0),
 ('B', 500),
 ('C', 125.0),
 ('E', 250.0),
 ('A', 250.0),
 ('C_high', 500),
 ('B', 500),
 ('E', 125.0),
 ('F', 250.0),
 ('C', 125.0),
 ('E', 250.0),
 ('A', 250.0),
 ('E', 125.0),
 ('C_high', 125.0),
 ('G', 250.0),
 ('B', 250.0),
 ('C', 250.0),
 ('F', 250.0),
 ('C', 500),
 ('C_high', 125.0),
 ('B', 250.0),
 ('B', 500),
 ('G', 125.0),
 ('E', 250.0),
 ('A', 125.0),
 ('D', 500)]

In [57]:

# Redefine the note to frequency conversion function
def note_to_freq(note):
    """ Convert musical note to its frequency, adjusted for rest. """
    note_frequencies = {
        'C': 261.63, 'D': 293.66, 'E': 329.63, 'F': 349.23, 
        'G': 392.00, 'A': 440.00, 'B': 493.88, 'C_high': 523.25
    }
    return note_frequencies.get(note, 0)  # Return 0 for rests or unknown notes


def melody_to_audio(melody):
    """ Convert a melody sequence to an audio segment. """
    song = AudioSegment.silent(duration=0)
    for note, duration in melody:
        frequency = note_to_freq(note)
        note_segment = Sine(frequency).to_audio_segment(duration=int(duration))
        song += note_segment

    return song

# Convert the provided melody sequence to audio
audio_output = melody_to_audio(melody_bigrams)

# Export to a WAV file
output_file = "./audios/bigramas_silencio.wav"
audio_output.export(output_file, format="wav")
output_file


'./audios/bigramas_silencio.wav'

# Imitando con la matriz de transición

Generamos 10 melodías

In [61]:
num_bars = 4
quarter_note_length = 500  # Quarter note length in milliseconds (0.5 seconds)

for i in range(10):
    melody_bigrams = generate_melody_bigrams(num_bars, quarter_note_length, transition_matrix_bigrams, bigram_states)
    # Convert the provided melody sequence to audio
    audio_output = melody_to_audio(melody_bigrams)

# Export to a WAV file
    output_file = f"./audios/melodias_candidatas/melodia_{i}.wav"
    audio_output.export(output_file, format="wav")
    

Tenemos que tomar las melodías, contar las transiciones entre cada nota, luego dividirla entre el total

In [176]:
states_with_sos_eos=['sos', 'C', 'D', 'E', 'F', 'G', 'A', 'B', 'C_high', 'eos']

states_with_duration_sos_eos=[]

for s in states_with_sos_eos:
    if s!='sos' and s!='eos':
        for duration in ['quarter','eighth','sixteenth']:
            si=s+'-'+duration
            states_with_duration_sos_eos.append(si)
    else:
        si=s 
        states_with_duration_sos_eos.append(si)
        
states_with_duration_sos_eos

['sos',
 'C-quarter',
 'C-eighth',
 'C-sixteenth',
 'D-quarter',
 'D-eighth',
 'D-sixteenth',
 'E-quarter',
 'E-eighth',
 'E-sixteenth',
 'F-quarter',
 'F-eighth',
 'F-sixteenth',
 'G-quarter',
 'G-eighth',
 'G-sixteenth',
 'A-quarter',
 'A-eighth',
 'A-sixteenth',
 'B-quarter',
 'B-eighth',
 'B-sixteenth',
 'C_high-quarter',
 'C_high-eighth',
 'C_high-sixteenth',
 'eos']

In [178]:
melody_bigrams

[('B', 125.0),
 ('E', 125.0),
 ('D', 500),
 ('E', 500),
 ('G', 250.0),
 ('C', 125.0),
 ('D', 125.0),
 ('F', 500),
 ('E', 125.0),
 ('A', 250.0),
 ('B', 500),
 ('C_high', 250.0),
 ('E', 125.0),
 ('G', 500),
 ('E', 250.0),
 ('A', 125.0),
 ('G', 500),
 ('F', 125.0),
 ('C_high', 250.0),
 ('A', 500),
 ('E', 500),
 ('D', 500),
 ('D', 500),
 ('F', 125.0),
 ('C_high', 125.0),
 ('G', 500)]

In [177]:

def melody_a_matriz(melody):
    
    count_matrix=np.zeros(shape=(len(states_with_duration_sos_eos),len(states_with_duration_sos_eos)))

    prev_state='sos'
    for note,duration in melody:
        mapeo={quarter_note_length:'quarter',quarter_note_length/2:'eighth', quarter_note_length/4:'sixteenth'}
        current_state=f'{note}-{mapeo[duration]}'

        prev_state_index=states_with_duration_sos_eos.index(prev_state)
        current_state_index=states_with_duration_sos_eos.index(current_state)
        
        count_matrix[prev_state_index,current_state_index]+=1
        # print(prev_state,current_state)
        prev_state=states_with_duration_sos_eos[current_state_index]
    return count_matrix

count_matrix=melody_a_matriz(melody_bigrams)
pd.DataFrame(count_matrix,index=states_with_duration_sos_eos,columns=states_with_duration_sos_eos)

,sos,C-quarter,C-eighth,C-sixteenth,D-quarter,D-eighth,D-sixteenth,E-quarter,E-eighth,E-sixteenth,F-quarter,F-eighth,F-sixteenth,G-quarter,G-eighth,G-sixteenth,A-quarter,A-eighth,A-sixteenth,B-quarter,B-eighth,B-sixteenth,C_high-quarter,C_high-eighth,C_high-sixteenth,eos
sos,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
C-quarter,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C-eighth,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C-sixteenth,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
D-quarter,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
D-eighth,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
D-sixteenth,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
E-quarter,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
E-eighth,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
E-sixteenth,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [121]:
num_bars = 4
quarter_note_length = 500  # Quarter note length in milliseconds (0.5 seconds)
count_matrix=np.zeros(shape=(len(states_with_duration_sos_eos),len(states_with_duration_sos_eos)))
for i in range(10):
    melody_bigrams = generate_melody_bigrams(num_bars, quarter_note_length, transition_matrix_bigrams, bigram_states)
    # Convert the provided melody sequence to audio
    audio_output = melody_to_audio(melody_bigrams)

# Export to a WAV file
    output_file = f"./audios/melodias_candidatas/melodia_{i}.wav"
    audio_output.export(output_file, format="wav")
    count_matrix+=melody_a_matriz(melody_bigrams)

In [138]:
pd.DataFrame(count_matrix,index=states_with_duration_sos_eos,columns=states_with_duration_sos_eos)

,sos,C-quarter,C-eighth,C-sixteenth,D-quarter,D-eighth,D-sixteenth,E-quarter,E-eighth,E-sixteenth,F-quarter,F-eighth,F-sixteenth,G-quarter,G-eighth,G-sixteenth,A-quarter,A-eighth,A-sixteenth,B-quarter,B-eighth,B-sixteenth,C_high-quarter,C_high-eighth,C_high-sixteenth,eos
sos,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
C-quarter,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0
C-eighth,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
C-sixteenth,0.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,6.0,0.0,0.0,1.0,0.0,0.0
D-quarter,0.0,1.0,0.0,0.0,1.0,0.0,0.0,3.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
D-eighth,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0
D-sixteenth,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
E-quarter,0.0,0.0,0.0,1.0,3.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
E-eighth,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
E-sixteenth,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0


In [137]:
def normalize_matriz(count_matrix):
    norm_matriz=np.zeros(shape=count_matrix.shape)
    for row in range(count_matrix.shape[0]):
        N=count_matrix[row,:].sum()
        if N!=0:
            norm_matriz[row]=count_matrix[row]/N 
    
    return norm_matriz

matriz_normalizada=normalize_matriz(count_matrix)
pd.DataFrame(matriz_normalizada,index=states_with_duration_sos_eos,columns=states_with_duration_sos_eos)

,sos,C-quarter,C-eighth,C-sixteenth,D-quarter,D-eighth,D-sixteenth,E-quarter,E-eighth,E-sixteenth,F-quarter,F-eighth,F-sixteenth,G-quarter,G-eighth,G-sixteenth,A-quarter,A-eighth,A-sixteenth,B-quarter,B-eighth,B-sixteenth,C_high-quarter,C_high-eighth,C_high-sixteenth,eos
sos,0.0,0.000000,0.000000,0.100000,0.000000,0.100000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.100000,0.000000,0.000000,0.100000,0.000000,0.100000,0.100000,0.000000,0.100000,0.100000,0.100000,0.000000,0.000000,0.0
C-quarter,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.200000,0.200000,0.000000,0.200000,0.0
C-eighth,0.0,0.000000,0.000000,0.125000,0.000000,0.125000,0.000000,0.000000,0.125000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.125000,0.000000,0.000000,0.000000,0.125000,0.0
C-sixteenth,0.0,0.000000,0.066667,0.000000,0.000000,0.133333,0.066667,0.000000,0.000000,0.000000,0.066667,0.000000,0.066667,0.000000,0.000000,0.066667,0.000000,0.000000,0.066667,0.000000,0.400000,0.000000,0.000000,0.066667,0.000000,0.0
D-quarter,0.0,0.076923,0.000000,0.000000,0.076923,0.000000,0.000000,0.230769,0.076923,0.000000,0.076923,0.000000,0.076923,0.076923,0.000000,0.153846,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.076923,0.000000,0.000000,0.0
D-eighth,0.0,0.083333,0.083333,0.083333,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.166667,0.083333,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.166667,0.000000,0.083333,0.083333,0.000000,0.0
D-sixteenth,0.0,0.000000,0.000000,0.142857,0.142857,0.000000,0.000000,0.142857,0.000000,0.000000,0.142857,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.285714,0.000000,0.0
E-quarter,0.0,0.000000,0.000000,0.062500,0.187500,0.000000,0.000000,0.062500,0.000000,0.000000,0.062500,0.062500,0.000000,0.000000,0.125000,0.000000,0.187500,0.000000,0.000000,0.000000,0.000000,0.062500,0.062500,0.062500,0.062500,0.0
E-eighth,0.0,0.000000,0.000000,0.083333,0.000000,0.083333,0.083333,0.083333,0.083333,0.166667,0.083333,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.083333,0.083333,0.083333,0.000000,0.000000,0.000000,0.000000,0.0
E-sixteenth,0.0,0.000000,0.000000,0.111111,0.111111,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.111111,0.000000,0.111111,0.000000,0.000000,0.111111,0.000000,0.000000,0.111111,0.111111,0.0


In [142]:
def generando_melodias_de_mat_transicion_long(matr_transicion,π0,size=100):
    nota_inicial=np.random.choice(a=states_with_duration_sos_eos,p=π0)

    notas=[nota_inicial]
    for i in range(size):
        indice_nota=states_with_duration_sos_eos.index(notas[-1])
        notas.append(np.random.choice(a=states_with_duration_sos_eos,p=matr_transicion[indice_nota]))
    
    return notas


In [144]:
π0=np.random.random(size=len(states_with_duration_sos_eos))
π0=π0/π0.sum()

melodia_1=generando_melodias_de_mat_transicion_long(matriz_normalizada,π0)


In [145]:


# Redefine the note to frequency conversion function
def note_to_freq(note):
    """ Convert musical note to its frequency. """
    note_frequencies = {
        'C': 261.63, 'D': 293.66, 'E': 329.63, 'F': 349.23, 
        'G': 392.00, 'A': 440.00, 'B': 493.88, 'C_high': 523.25
    }
    return note_frequencies.get(note, 0)  # Return 0 for rests or unknown notes

# Define the duration in milliseconds for a quarter note
quarter_note_duration_ms = 500  # Example duration

# Example array of notes with durations
note_array = [
    'B-quarter', 'E-quarter', 'D-quarter', 'D-quarter', 
    'C-quarter', 'B-sixteenth', 'E-sixteenth', 'D-sixteenth'
]

def array_to_audio(note_array, quarter_note_duration_ms):
    """ Convert an array of notes and durations to an audio segment. """
    song = AudioSegment.silent(duration=0)
    for item in note_array:
        note, duration_type = item.split('-')
        if duration_type == 'quarter':
            duration_ms = quarter_note_duration_ms
        elif duration_type == 'sixteenth':
            duration_ms = quarter_note_duration_ms / 4  # Sixteenth note is a quarter of a quarter note
        else:
            continue  # Skip unknown durations

        frequency = note_to_freq(note)
        note_segment = Sine(frequency).to_audio_segment(duration=int(duration_ms))
        song += note_segment

    return song

# Convert the note array to audio
audio_output = array_to_audio(melodia_1, quarter_note_duration_ms)

# Export to a WAV file
output_file = "./audios/array_melody.wav"
audio_output.export(output_file, format="wav")
output_file


'./audios/array_melody.wav'

# Ayudando a improvisación

El AI tocaría música como nosotros y nosotros 

# Música con Armonía



Haciendo acordes con distribuciones estacionarias

In [58]:
def calculate_transition_probabilities(melody, states):
    """Calculate the transition probabilities from the melody."""
    # Initialize the transition counts matrix
    transition_counts = {note: {next_note: 0 for next_note in states} for note in states}

    # Count transitions
    for i in range(len(melody) - 1):
        current_note = melody[i]
        next_note = melody[i + 1]
        transition_counts[current_note][next_note] += 1

    # Convert counts to probabilities
    transition_matrix = {note: {} for note in states}
    for note in states:
        total_transitions = sum(transition_counts[note].values())
        for next_note in states:
            if total_transitions > 0:
                transition_matrix[note][next_note] = transition_counts[note][next_note] / total_transitions
            else:
                transition_matrix[note][next_note] = 0

    return transition_matrix

# Example: A simple melody in C major (including silences)
# This is a simplified representation and may not reflect a true classical melody
example_melody = ['C', 'D', 'E', 'F', 'G', 'A', 'B', 'C_high', 'Rest', 
                  'E', 'D', 'C', 'Rest', 'G', 'F', 'E', 'D', 'C']

# Define the states (notes including silence)
states = ['C', 'D', 'E', 'F', 'G', 'A', 'B', 'C_high', 'Rest']

# Calculate the transition probabilities
transition_probs = calculate_transition_probabilities(example_melody, states)

# Display the transition probabilities matrix
transition_probs


{'C': {'C': 0.0,
  'D': 0.5,
  'E': 0.0,
  'F': 0.0,
  'G': 0.0,
  'A': 0.0,
  'B': 0.0,
  'C_high': 0.0,
  'Rest': 0.5},
 'D': {'C': 0.6666666666666666,
  'D': 0.0,
  'E': 0.3333333333333333,
  'F': 0.0,
  'G': 0.0,
  'A': 0.0,
  'B': 0.0,
  'C_high': 0.0,
  'Rest': 0.0},
 'E': {'C': 0.0,
  'D': 0.6666666666666666,
  'E': 0.0,
  'F': 0.3333333333333333,
  'G': 0.0,
  'A': 0.0,
  'B': 0.0,
  'C_high': 0.0,
  'Rest': 0.0},
 'F': {'C': 0.0,
  'D': 0.0,
  'E': 0.5,
  'F': 0.0,
  'G': 0.5,
  'A': 0.0,
  'B': 0.0,
  'C_high': 0.0,
  'Rest': 0.0},
 'G': {'C': 0.0,
  'D': 0.0,
  'E': 0.0,
  'F': 0.5,
  'G': 0.0,
  'A': 0.5,
  'B': 0.0,
  'C_high': 0.0,
  'Rest': 0.0},
 'A': {'C': 0.0,
  'D': 0.0,
  'E': 0.0,
  'F': 0.0,
  'G': 0.0,
  'A': 0.0,
  'B': 1.0,
  'C_high': 0.0,
  'Rest': 0.0},
 'B': {'C': 0.0,
  'D': 0.0,
  'E': 0.0,
  'F': 0.0,
  'G': 0.0,
  'A': 0.0,
  'B': 0.0,
  'C_high': 1.0,
  'Rest': 0.0},
 'C_high': {'C': 0.0,
  'D': 0.0,
  'E': 0.0,
  'F': 0.0,
  'G': 0.0,
  'A': 0.0,
  'B

## Dos cadenas de markov separadas

In [30]:
def generate_pitch_sequence(num_notes, transition_matrix_notes):
    """ Generate a sequence of pitches based on Markov chain. """
    pitch_sequence = []
    current_note = random.choice(notes)  # Starting note

    for _ in range(num_notes):
        pitch_sequence.append(current_note)
        current_note_index = notes.index(current_note)
        current_note = np.random.choice(notes, p=transition_matrix_notes[current_note_index])
    
    return pitch_sequence

def generate_duration_sequence(num_notes, transition_matrix_durations):
    """ Generate a sequence of durations based on Markov chain. """
    duration_sequence = []
    current_duration = random.choice(durations)  # Starting duration

    for _ in range(num_notes):
        duration_sequence.append(duration_lengths[current_duration])
        current_duration_index = durations.index(current_duration)
        current_duration = np.random.choice(durations, p=transition_matrix_durations[current_duration_index])
    
    return duration_sequence

# Example transition matrices for pitches and durations
transition_matrix_notes = np.full((len(notes), len(notes)), 1/len(notes))
transition_matrix_durations = np.full((len(durations), len(durations)), 1/len(durations))

# Generate pitch and duration sequences separately
num_bars = 4
quarter_note_length = 500  # Quarter note length in milliseconds (0.5 seconds)
total_quarters_in_piece = num_bars * 4  # Total number of quarter notes in the piece
num_notes = total_quarters_in_piece  # Assuming one note per quarter note for simplicity

pitch_sequence = generate_pitch_sequence(num_notes, transition_matrix_notes)
duration_sequence = generate_duration_sequence(num_notes, transition_matrix_durations)

# Combine pitch and duration sequences to form the melody
combined_melody = list(zip(pitch_sequence, [d * quarter_note_length for d in duration_sequence]))

combined_melody  # Displaying the combined melody (as note-duration pairs)


[('F', 125.0),
 ('F', 125.0),
 ('E', 250.0),
 ('C_high', 250.0),
 ('C', 125.0),
 ('G', 250.0),
 ('D', 500),
 ('C_high', 250.0),
 ('C', 125.0),
 ('B', 500),
 ('D', 125.0),
 ('F', 250.0),
 ('D', 125.0),
 ('F', 250.0),
 ('F', 500),
 ('C_high', 125.0)]

- Ver cómo ir agrupando estados para hacer algo cada vez menos Markoviano hasta llegar al punto en el que se toma en cuenta todo el estado previo.
- Hacer algo no estacionario.
- Ver distribución estacionaria de algún acorde y ponerlo en chinga para que suene armónico.
- Hacer clases de comunicación, estados absorbentes que sean escalas.
- Se le puede poner uno con ritmo, otro con melodía y otro con armonía
- Armar algo con refuerzo. 
    - Imitación del estilo de alguien.
    - Imitación de estilo supervisado por una persona.
- ¿Cómo pondrían algo menos predecible?
- Improvisación y cadenas de markov

Como la música no tiene una métrica tan clara de éxito o fracaso, crear cualquier programa que pueda fomentar la inspiración es muy útil. 

Los métodos sencillos ayudan a tener un entregable rápido que puede ayudar a detectar problemas futuros de un método más complicado

Poner varias melodías al mismo tiempo que convergen a la probabilidad estacionaria y ver cómo hacer cannons y cosas así

Empezar por uno que tenga sólo melodías a tiempos definidos, luego melodías con silencios, luego más notas, bigramas, etc.

Para cambiar la variedad, se puede solamente cambiar la varianza entre dos cosas

Con alguna api buscar un artista, sacar sus melodías y hacer música para improvisar.

- Armonía tradicional
- Algun tipo de modulación como clase de equivalencia